In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk
import zipfile

In [5]:
##jst unzipped the test data
local_zip='./test_data.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall()
zip_ref.close()

In [7]:
#unzipping the train data
local_zip='./vitamins_detection.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall()
zip_ref.close()

In [29]:
import os
base_dir='vitamins_detection'
print("contents of the base directory:")
print(os.listdir(base_dir))
vitaminA_train=os.path.join(base_dir,'vitaminAtr')
vitaminB_train=os.path.join(base_dir,'vitaminBtr')
vitaminC_train=os.path.join(base_dir,'vitaminCtr')
vitaminD_train=os.path.join(base_dir,'vitaminDtr')
vitaminE_train=os.path.join(base_dir,'vitaminEtr')
print('total training vitamin A images:',len(os.listdir(vitaminA_train)))
print('total training vitamin B images:',len(os.listdir(vitaminB_train)))
print('total training vitamin C images:',len(os.listdir(vitaminC_train)))
print('total training vitamin D images:',len(os.listdir(vitaminD_train)))
print('total training vitamin E images:',len(os.listdir(vitaminE_train)))

contents of the base directory:
['vitaminAtr', 'vitaminBtr', 'vitaminCtr', 'vitaminDtr', 'vitaminEtr']
total training vitamin A images: 1644
total training vitamin B images: 1704
total training vitamin C images: 3103
total training vitamin D images: 1104
total training vitamin E images: 1414


In [28]:
#now spilt the data as train and validation,(new chatgpt)
#totally forgot that they have give seperate dataset labeled as "test" taking that as validation
validation_dir='test_data'
print("contents of the validation directory")
print(os.listdir(validation_dir))
vitaminA_validation=os.path.join(validation_dir,'vitaminA')
vitaminB_validation=os.path.join(validation_dir,'vitaminB')
vitaminC_validation=os.path.join(validation_dir,'vitaminC')
vitaminD_validation=os.path.join(validation_dir,'vitaminD')
vitaminE_validation=os.path.join(validation_dir,'vitaminE')
print('total validation vitamin A images:',len(os.listdir(vitaminA_validation)))
print('total validation vitamin B images:',len(os.listdir(vitaminB_validation)))
print('total validation vitamin C images:',len(os.listdir(vitaminC_validation)))
print('total validation vitamin D images:',len(os.listdir(vitaminD_validation)))
print('total validation vitamin E images:',len(os.listdir(vitaminE_validation)))

contents of the validation directory
['vitaminA', 'vitaminB', 'vitaminC', 'vitaminD', 'vitaminE']
total validation vitamin A images: 52
total validation vitamin B images: 42
total validation vitamin C images: 62
total validation vitamin D images: 28
total validation vitamin E images: 40


In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#we need to create two data generators one for image training and one for validation
#declare the path to training dataset
train_dir='vitamins_detection'
#create a generator for training along with data augmentation,
train_datagen=ImageDataGenerator(rescale=1/255,rotation_range=45,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
#declare the path for validation
valid_dir='test_data'
valid_datagen=ImageDataGenerator(rescale=1/255)
#create generator
train_gen=train_datagen.flow_from_directory(train_dir,target_size=(224,224),class_mode='categorical',batch_size=15)
valid_gen=valid_datagen.flow_from_directory(valid_dir,target_size=(224,224),class_mode='categorical',batch_size=15)

Found 8969 images belonging to 5 classes.
Found 224 images belonging to 5 classes.


In [33]:
##model building! just copy and paste the import packages lines from skill wallet
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import softmax
from keras.api._v2.keras import activations
#now just import the vgg19 layer as said in skill wallet, 
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [35]:
#look the step initializing the model
Image_size=[224,224]
sol=VGG19(input_shape=Image_size+[3],weights='imagenet',include_top=False)
#to freeze trainable =freeze, fix the base model as false
sol.trainable = False

y=Flatten()(sol.output)
#the 19 layers itself is enough for feature detection, but let me add one more dense layer before the final output layer

y=Dense(255,activation='relu')(y)
final=Dense(5,activation='softmax')(y)
#softmax for classification

#declaring the model with input and output
model=Model(sol.inputs,final)

In [36]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [37]:
#we just made the model now lets compile and then fit
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['Accuracy'])
#optimizer like rmsprop can be used but as mentioned to use adam we use adam

In [38]:
#fitting the model and training it
model.fit(train_gen,epochs=30,validation_data=valid_gen,verbose=1)

Epoch 1/30
598/598 [==============================] - 1406s 2s/step - loss: 1.3969 - Accuracy: 0.4847 - val_loss: 1.0066 - val_Accuracy: 0.6116
Epoch 2/30
598/598 [==============================] - 1463s 2s/step - loss: 1.1216 - Accuracy: 0.5511 - val_loss: 0.9850 - val_Accuracy: 0.6518
Epoch 3/30
598/598 [==============================] - 1401s 2s/step - loss: 1.0805 - Accuracy: 0.5671 - val_loss: 0.9953 - val_Accuracy: 0.6429
Epoch 4/30
598/598 [==============================] - 1985s 3s/step - loss: 1.0486 - Accuracy: 0.5841 - val_loss: 0.8069 - val_Accuracy: 0.7411
Epoch 5/30
598/598 [==============================] - 1980s 3s/step - loss: 1.0298 - Accuracy: 0.5896 - val_loss: 0.9629 - val_Accuracy: 0.6786
Epoch 6/30
598/598 [==============================] - 1370s 2s/step - loss: 1.0145 - Accuracy: 0.5939 - val_loss: 0.8178 - val_Accuracy: 0.7188
Epoch 7/30
598/598 [==============================] - 1365s 2s/step - loss: 0.9921 - Accuracy: 0.6006 - val_loss: 0.9064 - val_Accuracy:

In [39]:
model

In [40]:
model.save('vitamin_model.h5')

In [41]:
model=tf.keras.models.load_model('vitamin_model.h5')

In [44]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions

In [80]:
img_path = 'carrots.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

In [81]:
prediction=model.predict(img_array)

1/1 [==============================] - 0s 193ms/step


In [82]:
prediction


array([[9.9993622e-01, 0.0000000e+00, 6.3755899e-05, 0.0000000e+00,
        6.3664985e-28]], dtype=float32)

In [83]:
predicted_class_index = np.argmax(prediction[0])

In [84]:
predicted_class_index

0

In [85]:
labels=["vitaminA","vitaminB","vitaminC","vitaminD","vitaminE"]
print(f'your item has {labels[predicted_class_index]}')

your item has vitaminA
